<a href="https://colab.research.google.com/github/gagomezuribe/03MIAR--Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Gustavo_Gomez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Gustavo Adolfo Gómez Uribe  <br>
Url Github: https://github.com/gagomezuribe/03MIAR--Algoritmos-de-Optimizacion <br>
Google Colab: https://colab.research.google.com/drive/1MqJ60qXGABhatzYjuYQCLQct116UNlJR?usp=sharing <br>
Problema:
**>1. Sesiones de doblaje <br>**

**Descripción del problema:**
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

Número de actores: 10; Número de tomas : 30; Actores/Tomas Cuadro en : https://bit.ly/36D8IuK 1=Participa en la toma








                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

**Espacio de soluciones:**
Una solución es un plan de grabación donde se indiquen las tomas que deben ser grabadas en cada día, con una restricción de un máximo de 6 tomas por día. El modelo de datos escogido para representar una solución consiste en una tupla de 30 valores, donde el índice corresponde al número de la toma y el valor al día en que dicha toma va a ser grabada.

Se debe cumplir las restricciones del problema para que una solución sea válida:
el dia debe estar entre 1 y el máximo, es decir, tomas total/tomas diarias, para este caso, 30/6 = 5, día pertenece [1,5] y no deben programarse más de 6 tomas por dia.

**Funcion objetivo:**
La función objetivo es la suma de costos de todos los dobles para la totalidad de las tomas, según la matriz de programación de actores por toma, teniendo en cuenta la restricción de que se pueden hacer un máximo de seis tomas por día.

El costo del día de doblaje por actor se consideró como una unidad. Dado que el tiempo de grabaciones es de 5 días y la cantidad de dobles es de 10, en una matriz en que todos los actores participaran en todas las tomas se tendría el costo máximo de 50. Con la matriz utilizada, el costo máximo es de 40 unidades de pago diarias. El objetivo del algoritmo es entonces minimizar esta función de costo, en unidades de pago diarias por doble.

**Implementación de las restricciones:**
Al crear una nueva solución debe validarse en el código que no existan mas de 6 tomas por día, es decir, que ningún día se repita más de 5 veces en la solución a analizar. La forma como se implementó en el algoritmo es partiendo de un vector modelo que cumpla las restricciones y asegurando que en cualquier cambio a la solución se conserve la condición de que la suma de tomas por día no excede el valor indicado (6)

#Análisis
**- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones**

Una forma de modelar el problema es mediante una tupla por cada día de grabacion, donde en cada tupla están las tomas que se van a realizar. Como se ilustrará a continuación, el orden de complejidad es factorial O(n!).

En el caso actual, al ser 30 tomas y un máximo de 6 tomas por día, se tendría que para el primer día hay 30!/(30-6)! es decir 30x29x28x27x26x25 = 4x10^8 combinaciones. En los siguientes días (dia 2 a día 5), siguiendo el mismo razonamiento, se tienen órdenes de 10^7, 10^7, 10^5 y 10^2 respectivamente, lo cual lleva a que el problema tiene un espacio de soluciones del orden de 10^29. Por esto no es viable usar métodos exactos y se debe acudir a metaheurísticas para buscar una solución razonablemente buena, sin saber si es la solución óptima.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Dado que la función de costo es muy sensible a pequeños cambios y a que no se puede construir una solución por pasos, los algoritmos evolutivos y genéticos son la mejor alternativa, pues parten de un universo amplio de posibles soluciones que se van ajustando con cambios leves hasta encontrar la mejor solución. Adicionalmente estos algoritmos son apropiados para que si existen mínimos locales se comparen diferentes soluciones y se pueda escoger la mejor, sin caer en un único mínimo local.

Para la búsqueda de la mejor solución se inició con un generador de soluciones aleatorias que cumplen la restricción de tener un máximo de 6 tomas por día.

Posteriormente se desarrolló un modelo de búsqueda local, con un cambio de 2-opt entre vecinos, cuya solución no fue muy buena.

Luego se desarrolló un algoritmo evolutivo partiendo de 500 soluciones con generaciones que eliminan 5 individuos en cada iteracion, hasta llegar a la mejor solución encontrada en 100 generaciones. Se encontraron mejores resultados con respecto al los algoritmos anteriores. Se hizo un análisis de sensibilidad a la probabilidad de la mutación, encontrando un mejor comportamiento con probabilidades bajas de mutación menores a 0.2

Finalmente se realizó un modelo de algoritmo genético, con un universo de 100 parejas, cada una generando cuatro hijos, de los cuales se escogía en cada generación los 50% menores, y aleatoriamente se cruzaban en una nueva iteracion.



In [2]:
#Librerías requeridas
import csv
import random
import numpy as np
from math import ceil


In [3]:
#Variables transversales, constantes y de entorno
global c_dias, c_actores, tomasxdia, tomas, arrmodelo, modeloini, dias_grab
c_actores,tomasxdia,ntomas = 10,6,30
c_dias = ceil(ntomas/tomasxdia)

#modelo básico de solucion donde se asigna un día de 1 a 5 a cada toma.
#se requiere esta lógica para permitir funcionamiento con programaciones de diferentes días
modeloini=[]
for i in range(1,c_dias+1):
  modeloini=modeloini+[i]*tomasxdia
arrmodelo = np.array(modeloini,dtype='uint8') #array modelo base, en numpy, con 30 elementos, uno por cada toma, cuyo valor es el dia de dicha toma
dias_grab = [x+1 for x in range(c_dias)] #lista de los días en que se pueden programar las grabaciones [1,2,3,4,5]

In [4]:
#Cargar el archivo de programación de tomas; el archivo contiene una primera columna del indice de las tomas y una última columna de totales. No tiene headers
ruta="/content/Datos problema doblaje.csv"
lista=[]
with open(ruta) as fichero:
    data_reader = csv.reader(fichero, delimiter=',')
    for linea in data_reader:
        lista.append(linea)
tomas= np.array(lista, dtype='uint8')
print(tomas)

[[ 1  1  1  1  1  1  0  0  0  0  0  5]
 [ 2  0  0  1  1  1  0  0  0  0  0  3]
 [ 3  0  1  0  0  1  0  1  0  0  0  3]
 [ 4  1  1  0  0  0  0  1  1  0  0  4]
 [ 5  0  1  0  1  0  0  0  1  0  0  3]
 [ 6  1  1  0  1  1  0  0  0  0  0  4]
 [ 7  1  1  0  1  1  0  0  0  0  0  4]
 [ 8  1  1  0  0  0  1  0  0  0  0  3]
 [ 9  1  1  0  1  0  0  0  0  0  0  3]
 [10  1  1  0  0  0  1  0  0  1  0  4]
 [11  1  1  1  0  1  0  0  1  0  0  5]
 [12  1  1  1  1  0  1  0  0  0  0  5]
 [13  1  0  0  1  1  0  0  0  0  0  3]
 [14  1  0  1  0  0  1  0  0  0  0  3]
 [15  1  1  0  0  0  0  1  0  0  0  3]
 [16  0  0  0  1  0  0  0  0  0  1  2]
 [17  1  0  1  0  0  0  0  0  0  0  2]
 [18  0  0  1  0  0  1  0  0  0  0  2]
 [19  1  0  1  0  0  0  0  0  0  0  2]
 [20  1  0  1  1  1  0  0  0  0  0  4]
 [21  0  0  0  0  0  1  0  1  0  0  2]
 [22  1  1  1  1  0  0  0  0  0  0  4]
 [23  1  0  1  0  0  0  0  0  0  0  2]
 [24  0  0  1  0  0  1  0  0  0  0  2]
 [25  1  1  0  1  0  0  0  0  0  1  4]
 [26  1  0  1  0  1  0  0

In [5]:
#Función para crear una solución aleatoria; recibe y devuelve ndarray
def crear_solucion_rnd(sol_ini=arrmodelo):

  random.shuffle(sol_ini)
  return sol_ini

print(crear_solucion_rnd())

[1 3 3 2 4 3 4 5 1 5 5 4 1 4 2 4 1 3 2 1 5 3 2 5 2 1 5 3 2 4]


In [6]:
#Función para calcular el costo de una solución
def costo(sol):  #parámetro es ndarray con la solución a la que se le calculará el costo
  #Cálculo de la matriz de programación: días en que trabaja cada actor
  matrizprog = np.zeros((30,11),dtype='uint8')
  for i in range(1,11):
   matrizprog[:,i-1] = np.multiply(sol,tomas[:,i])

  #cálculo de vector de dias por actor:
  dias_actor = np.zeros(10,dtype='uint8') #cada índice corresponde a un actor o doble. El valor es la cantidad de días que debe trabajar dicho doble [1-5]
  for i in range(0,10):
    dias_actor[i] = len(np.unique(matrizprog[:,i]))-1 #para cada actor se suman los días en que ha tenido que participar según la programación de tomas
  return np.sum(dias_actor)


In [103]:
#BUSQUEDA ALEATORIA
#cambia al azar la solución, hace iteraciones y escoge la mejor solución encontrada
def busqueda_aleatoria():
  costo_min = 100000   #se inicializa alto para que se ajuste en la primera iteración
  mejor_sol=[]
  for i in range(0,30):
    sol_i = crear_solucion_rnd()
    costo_i = costo(sol_i)
    if costo_i <costo_min:
      costo_min = costo_i
      mejor_sol = sol_i.copy()
  return mejor_sol,costo_min

sol= busqueda_aleatoria()[0]
print("Mejor solución: ", sol)
print("Costo: ",costo(sol))


Mejor solución:  [3 2 5 5 3 3 3 4 4 4 3 1 2 4 2 5 4 1 1 3 1 2 5 1 2 2 1 5 4 5]
Costo:  36


# BUSQUEDA LOCAL

In [110]:
#BUSQUEDA LOCAL
#Toma la mejor opción a partir de una búsqueda aleatoria y haciendo todos los cambios posibles 2-opt busca una mejor solución
mejor_sol, costo_min = busqueda_aleatoria()
print("Al iniciar búsqueda local:")
print(mejor_sol,costo_min)
# print(mejor_sol,costo_min)
for i in range(ntomas):
  for j in range(i+1,ntomas):
    temp = mejor_sol[i]
    sol_i = mejor_sol
    sol_i[i]=sol_i[j]
    sol_i[j]=temp
    costo_i = costo(sol_i)
    if costo_i <costo_min:
      costo_min = costo_i
      mejor_sol = sol_i.copy()
print("Luego de búsqueda local:")
print(mejor_sol,costo_min)

Al iniciar búsqueda local:
[4 1 2 5 2 5 1 3 2 4 2 5 4 2 1 5 3 3 1 4 5 1 1 3 2 4 4 5 3 3] 37
Luego de búsqueda local:
[3 3 5 4 4 2 3 1 1 5 4 1 3 3 5 1 2 4 5 2 4 2 3 1 5 2 5 2 1 4] 33


# ALGORITMO EVOLUTIVO

In [90]:
#ALGORITMO EVOLUTIVO
#Se parte de un número de soluciones. A partir de ellas, se selecciona el 90% de los mejores individuos, se hace una mutacion aleatoria y se repite hasta finalizar
#Estructura de datos: espacio de soluciones es un ndarray bidimensional, donde las filas son las soluciones y las columnas las tomas; los valores, el día de la toma

#DESCARTAR: descartar las peores n soluciones. Para esto se crea una lista con el identificador de la solución  y su costo. luego se ordena
def descarta_n_peores(espacio_soluciones,n):
  ranking = np.zeros((np.shape(espacio_soluciones)[0],2))
  for s in range(np.shape(espacio_soluciones)[0]):
    ranking[s,0]=s
    ranking[s,1]=costo(espacio_soluciones[s])

  #Ordenar según el costo, para eliminar los de mayor costo:
  indices_ordenados = np.argsort(ranking[:, 1])
  espacio_soluciones = espacio_soluciones [indices_ordenados]

  #Eliminar los de mayor costo, las últimas n filas:
  espacio_soluciones = espacio_soluciones[:-n,:]
  return espacio_soluciones


#MUTAR: recorre todas las soluciones del espacio y con una probabilidad cambia al azar un elemento de la solución, manteniendo la restricción:
def muta_el_resto(espacio_soluciones):
  espacio_soluciones_output = np.copy(espacio_soluciones)
  for s in range(np.shape(espacio_soluciones)[0]):
    if random.random()>0.8:
      ind_elemento_mutar = random.randint(0,ntomas-1) # se selecciona uno de los elementos (índice) de la solución (una toma)
      dia_cambiar =espacio_soluciones[s][ind_elemento_mutar] # se obtiene el día planeado en la solución para grabar
      dia_nuevo = random.sample([x for x in dias_grab  if x != dia_cambiar],1 )[0] #se selecciona al azar un día diferente
      lista_posibles = np.where(espacio_soluciones[s]==dia_nuevo) #se obtiene de la solución en análisis las tomas planeadas para el día nuevo
      ind_elemento_cambiar = np.random.choice(lista_posibles[0],size=1)
      espacio_soluciones_output[s,ind_elemento_mutar] = dia_nuevo
      espacio_soluciones_output[s,ind_elemento_cambiar] = dia_cambiar
  return espacio_soluciones_output


#PROGRAMA PRINCIPAL DEL ALGORITMO EVOLUTIVO (EA)
#Inicializar la lista de soluciones:
cant_individuos=501
espacio_soluciones = np.zeros((cant_individuos,ntomas),dtype='uint8') # espacio_soluciones es un ndarray

for j in range(cant_individuos):
 sol = crear_solucion_rnd()
 espacio_soluciones[j]=sol

#Iteraciones mutación - evolución (generaciones) hasta dejar una solución final
while np.shape(espacio_soluciones)[0]>5:
  espacio_soluciones = descarta_n_peores(espacio_soluciones,5) #descarta 5 soluciones cada iteración
  espacio_soluciones = muta_el_resto(espacio_soluciones) #con un función probabilística, cambia una de las tomas, en analogía con una mutación

#presentar la mejor solución encontrada:
for s in range(np.shape(espacio_soluciones)[0]):
  print(espacio_soluciones[s])
  print(costo(espacio_soluciones[s]))



[1 5 4 4 4 1 4 5 3 2 2 5 2 5 3 3 1 2 1 2 5 1 5 3 3 2 4 1 3 4]
34


# ALGORITMO GENETICO

In [85]:
from os import pardir
#ALGORITMO GENETICO
#Se parte de un número de soluciones, organizadas en parejas, para efectos del cruce.
#Cada pareja se cruza: se toma uno de los padres y se cambia un gen (día de programación) por el del otro progenitor
#El hijo se ajusta a la restricción, asegurando que esta se cumpla
#Se escoge el 50% mejor de los invididuos de la generación  (seleccion por "torneo")
#Con toda la nueva generación se organizan nuevamente el mismo conjunto de parejas y se repite el cruce
#Se itera el proceso un numero definido de iteraciones


#Función de cruce para generar hijos de la primera pareja (provisional)
def generar_hijos(padres,madres):
  hijos = np.zeros((cant_hijos*cant_parejas,ntomas),dtype='uint8')
  i=0

  for p in range(cant_parejas):
    for h in range(cant_hijos): #bucle para crear los n hijos en un ndarray n x tomas
      gen = random.randint(0,ntomas-1) #escoger el gen a intercambiar
      prog = random.randint(0,1) #escoger el progenitor base: padre o madre

      if prog == 0:
        progenitor_base = np.copy(padres[p])
        gen_cambiado = madres[p,gen]
      else:
        progenitor_base = np.copy(madres[p])
        gen_cambiado = padres[p,gen]
      gen_original = progenitor_base[gen]
      progenitor_base[gen] = gen_cambiado #sustituye un gen por el del otro progenitor

      #ajustar el hijo para que cumpla la restricción:
      indices_compensar = np.where(progenitor_base == gen_cambiado)
      #de la lista de los dias programados en exceso se escoge uno al azar para compensar:
      ix = int(np.random.choice(indices_compensar[0],1)[0] )
      while ix == gen: #tomar otro de la lista si se escoge el índice del gen cambiado
        ix = int(np.random.choice(indices_compensar[0],1)[0] ) #al azar entre los candidatos
      progenitor_base[ix] = gen_original #mantener la restricción
      hijos[i] = progenitor_base
      i+=1

  return hijos

#Función para implementar el torneo: ordena la generación, escoge el 50% mejor
def ordenar_generacion(nueva_generacion): #recibe un ndarray filas son la cantidad de hijos, columnas las tomas
    ranking = np.zeros((np.shape(nueva_generacion)[0],2)) #ranking es un array temporal con el costo de cada solución
    for s in range(np.shape(nueva_generacion)[0]): #calcula y almacena el costo de cada solución de la nueva generación
      ranking[s,0]=s
      ranking[s,1]=costo(nueva_generacion[s])
    indices_ordenados = np.argsort(ranking[:, 1]) #ordenar según el costo
    nueva_generacion = nueva_generacion [indices_ordenados]
    nueva_generacion = nueva_generacion[:cant_parejas*2,:] #borra el 50% de individuos de la generacion con mayor costo
    return(nueva_generacion)

def nuevas_parejas(nueva_generacion):
  i=0 #inicializa el contador para indexar las parejas
  padres = np.zeros((np.shape(nueva_generacion)[0]//2,ntomas),dtype='uint8')
  madres = np.zeros((np.shape(nueva_generacion)[0]//2,ntomas),dtype='uint8')

  while np.shape(nueva_generacion)[0]>1:
    par=random.sample(range(np.shape(nueva_generacion)[0]),2)
    padres[i]=nueva_generacion[par[0]]
    madres[i]=nueva_generacion[par[1]]
    i+=1
    nueva_generacion = np.delete(nueva_generacion,par,axis=0) #borra del espacio las dos solucioens que se convirtieron en progenitoras

  return (padres,madres)


#INICIO MODULO PRINCIPAL
#GENERAR PAREJAS:
#Estructura de datos: Padres un ndarray bidimensional, donde las filas son las soluciones y las columnas las tomas; los valores, el día de la toma

cant_parejas = 100
cant_hijos = 4
padres = np.zeros((cant_parejas,ntomas),dtype='uint8')
madres = np.zeros((cant_parejas,ntomas),dtype='uint8')
hijos = np.zeros((cant_hijos,ntomas),dtype='uint8')
nueva_generacion = np.zeros((cant_hijos*cant_parejas,ntomas),dtype='uint8')

for j in range(cant_parejas):
 padres[j]=crear_solucion_rnd()
 madres[j]=crear_solucion_rnd()

#Bucle para barrer n generaciones
for g in range(100):
    #Bucle para generar hijos de c/u de las parejas:
    nueva_generacion=generar_hijos(padres,madres) #la funcion devuelve un ndarray de filas= cant_parejas x cant_hijos por pareja
    nueva_generacion = ordenar_generacion(nueva_generacion) #ordena la generación según el costo, escoge los 50 mejores; retorna el nuevo espacio
    padres,madres = nuevas_parejas(nueva_generacion) #y prepara padres y madres

#Escoger la mejor_solucion de la ultima generacion:
costo_min=10000
for sol in nueva_generacion:
 if costo(sol)<costo_min:
  mejor_sol = sol
  costo_min = costo(sol)

print("Mejor solución encontrada: ",mejor_sol)
print("Costo de la mejor solución: ",costo_min)

Mejor solución encontrada:  [5 5 3 3 3 3 3 1 2 1 1 5 3 4 2 2 4 4 4 5 1 5 4 4 2 1 5 2 1 2]
Costo de la mejor solución:  27


# CONCLUSION

En síntesis, se puede apreciar que la escogencia de la metaheuristica es de vital importancia para encontrar buenas soluciones. A pesar de que en este problema el rango de diferencias de costo es pequeño, se pudieron notar importantes diferencias entre los resultados conseguidos con las diferentes alternativas analizadas.


La búsqueda aleatoria no da resultados muy buenos. Se obtuvieron valores entre 35 y 40 unidades diarias de salario


La búsqueda local y el algoritmo evolutivo tuvieron mejores resultados, siendo el mejor valor en diferentes ejecuciones 33 unidades diarias de salario


La mejor heuristica para este problema, tal como se indicó en la parte de análisis y escogencia del modelo, es el algoritmo genético, pues se distribuyen las soluciones en diferentes partes del espacio de soluciones y se optimiza con gradualidad de generación en generacion mediante los cruces y elitismo, hasta encontrar una buena solución. Los parámetros utilizados para lograr un balance entre tiempo de ejecución y calidad del resultado fueron 100 parejas, que generan 4 hijos cada una, y que iteran por 100 generaciones, con un tiempo de 10 segundos y un resultado de costo 27, mucho mejor que el que se obtuvo con los demás métodos.